<a href="https://colab.research.google.com/github/denmj/Algos-Euler/blob/master/keras_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
print("conda env: ", os.environ['CONDA_DEFAULT_ENV'])
print("conda env path: ", os.environ['CONDA_PREFIX'])


conda env:  algo-euler-conda
conda env path:  C:\Users\dmamed01\Anaconda3\envs\algo-euler-conda


In [2]:
import numpy as np
import pandas as pd
import scipy as sp

In [ ]:
import pandas as pd
from matplotlib import pyplot

from numpy import loadtxt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import LSTM

from tensorflow.keras.utils import plot_model

from tensorflow.keras.models import model_from_json, model_from_yaml, load_model
import numpy as np
import os

In [ ]:
def parser(x):
	return pd.datetime.strptime('190'+x, '%Y-%m')

In [ ]:
dataset = pd.read_csv('/content/shampoo_sales.csv',  header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [ ]:
dataset

In [ ]:
dataset.plot()
pyplot.show()

In [ ]:

# split data into train and test
X = dataset.values
train, test = X[0:-12], X[-12:]

In [ ]:
hist = [x for x in train]

In [ ]:
test

In [ ]:
hist[-1]

In [ ]:
predictions = list() 

In [ ]:
for i in range(len(test)):
  predictions.append(hist[-1])
  hist.append(test[i])

In [ ]:
predictions

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error

rmse = sqrt(mean_squared_error(test, predictions))
print('RMSE: %.3f' % rmse)

In [ ]:
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

In [ ]:
lag =1 

In [ ]:
df = pd.DataFrame(train)

In [ ]:
columns = [df.shift(i) for i in range(1, 2)]

In [ ]:
columns

In [ ]:
columns.append(df)

In [ ]:
columns

In [ ]:
df = pd.concat(columns, axis=1)

In [ ]:
df

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df

In [ ]:

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

In [ ]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#transforming 

X = dataset.values
X = X.reshape(len(X), 1)


In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
scaler = scaler.fit(X)

In [ ]:
scaler_X = scaler.transform(X)

In [ ]:
inverted_x = scaler.inverse_transform(scaler_X)

In [ ]:
# LSTM Development (RNN - Reccurrent Neural Network)

In [ ]:
# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
 
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df
 
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]
 
# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = np.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

In [ ]:
series = pd.read_csv('/content/shampoo_sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)


In [ ]:
# TRANSFORMATION STEPS 

In [ ]:
# GET DIFFERENCE FROM DATA (STATIONARY PART)

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return pd.Series(diff)

In [ ]:
raw_values = series.values

In [ ]:
diff_values = difference(raw_values, 1)

In [ ]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = pd.DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = pd.concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

In [ ]:
supervised = timeseries_to_supervised(diff_values, 1)

In [ ]:
supervised_values  =  supervised.values

In [ ]:
train, test = supervised_values[0:-12], supervised_values[-12:]

In [ ]:
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

In [ ]:
scaler, train_scaled, test_scaled = scale(train, test)

In [ ]:
# Basic LSTM 


In [ ]:
X, y  = train_scaled[:, 0:-1], train_scaled[:, -1]

In [ ]:
X = X.reshape(X.shape[0], 1, X.shape[1])

In [ ]:
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, shuffle=False)
		model.reset_states()
	return model

In [ ]:
model = fit_lstm(train_scaled, 1, 1500, 1)

In [ ]:
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)

In [ ]:
model.predict(train_reshaped, batch_size=1)

In [ ]:
test_scaled

In [ ]:
predictions = list() 

In [ ]:
for i in range(len(test_scaled)):
  X, y = test_scaled[i, 0:-1], test_scaled[i, -1] 
  print(i)

In [ ]:
test_point = test_scaled[1, 0:-1]
test_y = test_scaled[1, -1]

In [ ]:
tp = test_point.reshape(1, 1, len(test_point))

In [ ]:
test_point, tp

In [ ]:
pred_point = model.predict(tp, batch_size=1)

In [ ]:
pred_point, test_y

In [ ]:
y_h = invert_scale(scaler, test_point, pred_point)

In [ ]:
y_h